In [135]:
import feedparser
from collections import defaultdict
import hashlib
from threading import Timer
import time
from pandas import DataFrame
import datetime
import pandas as pd
import os.path

dump_file = "news.csv"
# setup scheduler
s = sched.scheduler(time.time, time.sleep)

def open_dump(path):
    if os.path.isfile(path):
        return pd.read_csv(path)

def scrap(hashes):
    ts = int(time.time())
    dt = datetime.datetime.now()    
    data_temp = []
    feed_raw = feedparser.parse('https://fivethirtyeight.com/features/feed/')
    feed_entries = feed_raw['entries']

    for f in feed_entries:
        title = f['title'].encode('utf-8')
        hashed_title = hashlib.md5(title).hexdigest()

        if not hashed_title in hashes:
            entry = {'dt': dt, 'ts' : ts, 'title' :  title, 'hash': hashed_title} 
            data_temp.append(entry)

    df = DataFrame.from_records(data_temp, columns = ['dt', 'ts', 'title', 'hash']) 
    #df.columns
    return df

def run_dump():
    data = open_dump(dump_file)
    if data is None or data.empty:
        print "No CSV dump"
        # first start without csv
        res = scrap([])
        res.to_csv(path_or_buf=dump_file, columns = ['dt', 'ts', 'title', 'hash'])
    else:
        hashes = data['hash'].tolist()
        res = scrap(hashes)
        data = data.append(res, ignore_index=True)
        data.to_csv(path_or_buf=dump_file, columns = ['dt', 'ts', 'title', 'hash'])
    Timer(10, run_dump).start()

def run():
    run_dump()        
    

In [134]:
run_dump()

In [ ]:
def read_dump():
    return op